# SHELFIE

Run all neccessary pip installs

In [ ]:
%pip uninstall -qqy jupyterlab 
# Remove unused conflicting packages
%pip install -U -q "google-genai==1.7.0"
%pip install faker
%pip install google-api-python-client
%pip install dotenv

Do all the neccessary imports to declutter code below. Also verify genai is being imported correctly.

In [ ]:
import sqlite3
import random
from faker import Faker
from google import genai
from google.genai import types
from google.api_core import retry
from typing import Final
from IPython.display import Markdown, display
from pprint import pprint
print(genai.__version__)

Set up `GOOGLE_API_KEY` for both Kaggle and local development.

In [ ]:
import os
from dotenv import load_dotenv

try:
    from kaggle_secrets import UserSecretsClient
    IS_KAGGLE = True
except ImportError:
    IS_KAGGLE = False

# Load environment variables from .env file for local setup
if not IS_KAGGLE:
    load_dotenv()

# Fetch the Google API key
if IS_KAGGLE:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
else:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Please set it in the appropriate environment.")

In [ ]:
NUM_BOOKS: Final[int] = 30

In [ ]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
#SYSTEM_INSTRUCTION: "You are Shelfie, a library chatbot facilitating book borrowing through this chat interface. Your knowledge encompasses all books available in our library, including detailed understanding of plots, characters, themes, and precise recall of passages with corresponding page and book information. You can also provide a rating out of 5 for each book. When a requested book is unavailable, offer recommendations for highly-rated alternatives within our current inventory."

In [ ]:
# Remove existing database file (if it exists)
db_file = "library.db"
if os.path.exists(db_file):
    os.remove(db_file)

In [ ]:
from dataclasses import dataclass

@dataclass
class Book:
    name: str
    genre: str
    description: str
    author: str

In [ ]:
def generate_books_list(num_books: int) -> list[Book]:
    """
    Asks Gemini to create a list of
    
    return: A list of Book objects.
    """
    prompt = """
    Generate a list of exactly {num_books} books of any genre. For each book, provide the name, genre, a description of about 1000 characters, and the author.
    Do not include the author's name in the description. Return the result as a JSON array of book objects with the keys: "name", "genre", "description", and "author".
    Examples of a "name", "genre", "description", and "author" are provided below:
    1) "To Kill a Mockingbird", "Fiction", "A gripping tale set in the racially charged American South, exploring themes of justice, morality, and the loss of innocence. Perfect for readers who appreciate profound social commentary interwoven with compelling storytelling.", "Harper Lee"
    2) "1984", "Dystopian", "A chilling vision of a totalitarian future where surveillance and control pervade every aspect of life. Ideal for fans of thought-provoking and cautionary tales.", "George Orwell"
    """

    if num_books < 1:
        raise Exception("Need to generate at least one Book.")
    
    prompt = prompt.format(num_books = num_books)
    client = genai.Client(api_key=GOOGLE_API_KEY)
    
    structured_output_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema={
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "genre": {"type": "string"},
                    "description": {"type": "string"},
                    "author": {"type": "string"}
                },
                "required": ["name", "genre", "description", "author"]
            }
        },
        temperature=0.3
    )

    response_len = 0

    while response_len != num_books:
        chat = client.chats.create(model='gemini-2.0-flash')
        response = chat.send_message(
          message=prompt,
          config=structured_output_config,
        )
        structured_eval = response.parsed
        book_objects = [Book(**book) for book in response.parsed]
        response_len = len(book_objects)
    
    return book_objects

Do a basic check to see that an exception is raised when the user provides `num_books` < 1.

In [ ]:
try:
    generate_books_list(0)
except:
    print("Expected error when calling generate_books_list with 0 books.\n\n")

Call `generate_books_list` with `NUM_BOOKS`. The time it takes to run the line of code is non-deterministic. This is the number of books produced by gemini is not consitent.

In [ ]:
books_list = generate_books_list(NUM_BOOKS)

Do some validation on the generated output. Verify if the generated list is indeed of length 30. If it's not try running it again. Print the top 5 for inspection.

In [ ]:
assert(len(books_list) == NUM_BOOKS)
assert(all([True for book in books_list if type(book) == Book]))
pprint(books_list[:5])

In [ ]:
# Initialize Faker instance
fake = Faker()

# Create SQLite database and tables
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    password_hash TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Book (
    book_id INTEGER PRIMARY KEY,
    name TEXT,
    genre TEXT,
    description TEXT,
    author TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Stock (
    stock_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS BorrowingHistory (
    entry_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    stock_id INTEGER,
    borrowed_on DATE,
    returned_on DATE,
    due_by DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (stock_id) REFERENCES Stock(stock_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Reservations (
    reservation_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    book_id INTEGER,
    status TEXT,
    last_updated DATE,
    reserved_on DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Waitlist (
    book_id INTEGER,
    waitlisted_on DATE,
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
)
""")

passwords = ["ISNA2018", "ISNA2019", "ISNA2020", "ISNA2021", "ISNA2022"]
# Generate and insert sample data for Users table
for user_id in range(1, 6):
    cursor.execute("""
    INSERT INTO Users (user_id, name, email, password_hash)
    VALUES (?, ?, ?, ?)
    """, (user_id, fake.name(), fake.email(), passwords[user_id - 1]))

# Insert real books into Book table
for book_id, book in enumerate(books_list, start=1):
    cursor.execute("""
    INSERT INTO Book (book_id, name, genre, description, author)
    VALUES (?, ?, ?, ?, ?)
    """, (book_id, book.name, book.genre, book.description, book.author))

# Generate and insert sample data for Stock table
for book_id in range(1, 31):
    num_books = random.randint(3, 5)
    j = 0
    while (j < num_books):
        cursor.execute("""
        INSERT INTO Stock (book_id)
        VALUES (?)
        """, (book_id, ))
        j += 1

# Generate and insert sample data for BorrowingHistory table
for entry_id in range(1, 21):
    cursor.execute("""
    INSERT INTO BorrowingHistory (entry_id, user_id, stock_id, borrowed_on, returned_on, due_by)
    VALUES (?, ?, ?, ?, ?)
    """, (entry_id, random.randint(1, 5), random.randint(1, 50), fake.date_between(start_date="-1y", end_date="-1d"), fake.date_between(start_date="+1d", end_date="+1y"), fake.date_between(start_date="+1d", end_date="+1y")))

# Generate and insert sample data for Reservations table
for reservation_id in range(1, 21):
    cursor.execute("""
    INSERT INTO Reservations (reservation_id, user_id, book_id, status, last_updated, reserved_on)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (reservation_id, random.randint(1, 5), random.randint(1, 50), random.choice(["Active", "Fulfilled", "Expired"]), fake.date_between(start_date="-1y", end_date="today"), fake.date_between(start_date="-1y", end_date="today")))

# Generate and insert sample data for Waitlist table
for book_id in range(1, 6):
    cursor.execute("""
    INSERT INTO Waitlist (book_id, waitlisted_on)
    VALUES (?, ?)
    """, (book_id, fake.date_between(start_date="-1y", end_date="today")))

# Commit changes and close connection
conn.commit()
conn.close()

print("Sample data has been inserted into the SQLite database.")


In [ ]:
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

# Query all tables to display the data
cursor.execute("SELECT * FROM Users")
print("Users:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Book")
print("\nBooks:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Stock")
print("\nStock:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM BorrowingHistory")
print("\nBorrowingHistory:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Reservations")
print("\nReservations:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM Waitlist")
print("\nWaitlist:")
for row in cursor.fetchall():
    print(row)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session